In [1]:
from flask import Flask, request, jsonify, render_template, session, send_file, redirect, url_for
from flask_session import Session
from openpyxl import load_workbook
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
from io import BytesIO  # Import BytesIO for in-memory binary streams
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

app = Flask(__name__)
# Configure the Flask app to use server-side session
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)

app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///bids.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

def load_sheets(filename):
    workbook = load_workbook(filename)
    inventory = []
    factors = []

    # Load Inventory Sheet
    inventory_sheet = workbook["Inventory"]  # Use the correct name of your Inventory sheet
    inventory_headers = [cell for cell in next(inventory_sheet.iter_rows(min_row=1, max_row=1, values_only=True))]
    for row in inventory_sheet.iter_rows(min_row=2, values_only=True):
        item = {inventory_headers[i]: value for i, value in enumerate(row)}
        inventory.append(item)
    
    # Load Factor Sheet
    factor_sheet = workbook["Factor"]  # Use the correct name of your Factor sheet
    factor_headers = [cell for cell in next(factor_sheet.iter_rows(min_row=1, max_row=1, values_only=True))]
    for row in factor_sheet.iter_rows(min_row=2, values_only=True):
        item = {factor_headers[i]: value for i, value in enumerate(row)}
        factors.append(item)
    
    return inventory, factors

inventory, factors = load_sheets('Input.xlsx')



@app.route('/new-bid')
def new_bid():
    session.setdefault('bid_items', [])
    total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
    return render_template('Bid_Job_Estimating.html', bid_items=session['bid_items'], total_cost=total_cost)


@app.route('/')
def home():
    # Basic homepage with links to other parts of the application
    return render_template('Home.html')

@app.route('/add', methods=['POST'])
def add_item():
    part_num = request.form['PartNum']
    quantity = int(request.form['Quantity'])
    item = next((item for item in inventory if item['PartNum'] == part_num), None)
    if item:
        # Append to bid_items in session
        session['bid_items'].append({**item, "Quantity": quantity, "Cost": float(item['Cost'])})
        session.modified = True  # Mark the session as modified to save changes
        total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
        return jsonify(success=True, bid_items=session['bid_items'], total_cost=total_cost)
    else:
        return jsonify(success=False)

@app.route('/delete', methods=['POST'])
def delete_items():
    selected_items = request.json['selectedItems']
    # Assuming 'bid_items' is stored in session and is a list of dictionaries
    session['bid_items'] = [item for item in session.get('bid_items', []) if item['PartNum'] not in selected_items]
    total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
    return jsonify({'bid_items': session['bid_items'], 'total_cost': total_cost})

@app.route('/inventory')
def get_inventory():
    return jsonify(inventory)

@app.route('/factors')
def get_factors():
    # Assuming factors is a list of dictionaries
    try:
        # Debug print to inspect the structure
        print(factors)
        return jsonify(factors)
    except TypeError as e:
        # Log the error for debugging
        print(f"Error serializing factors: {e}")
        # Return an error message or empty list as a fallback
        return jsonify([]), 500

@app.route('/add-update-bid', methods=['GET', 'POST'])
def add_update_bid():
    if request.method == 'POST':
        create_new_bid = request.form.get('CreateNewBid') == 'on'
        bid_name = request.form.get('BidName')
        
        if create_new_bid:
            # Check if a bid with the same name already exists
            existing_bid = Bid.query.filter_by(name=bid_name).first()
            if existing_bid is None:
                # Create new bid and add it to the database
                new_bid = Bid(name=bid_name)
                db.session.add(new_bid)
                db.session.commit()
                # Assuming you have a way to pass the created bid's ID or name to the new bid page
                session['current_bid_id'] = new_bid.id
                return redirect(url_for('new_bid'))
            else:
                # Handle the case where a bid with the same name already exists
                return "A bid with this name already exists.", 400
        else:
            # Handle updating an existing bid, if applicable
            pass
    else:
        return render_template('AddUpdateBid.html')
@app.route('/create-proposal')
def create_proposal():
    next_tab = request.args.get('next_tab', None)
    return render_template('create_proposal.html', next_tab=next_tab)
 

@app.route('/create-proposal_report')
def create_proposal_report():
    proposal_data = session.get('proposal_data', {})
    buffer = BytesIO()
    c = canvas.Canvas(buffer, pagesize=letter)
    # Adjust y as needed
    y = 750
    
    # Use proposal_data to draw text on the PDF
    c.drawString(100, y, f"Bid ID: {proposal_data.get('bidID', 'N/A')}")
    # Continue for other fields
    
    c.save()
    buffer.seek(0)
    return send_file(buffer, as_attachment=True, download_name='proposal.pdf', mimetype='application/pdf')



@app.route('/submit-proposal-data', methods=['POST'])
def submit_proposal_data():
    # Store or update proposal data in session
    session['proposal_data'] = request.form.to_dict()
    
    # Extract the next tab from the form data
    next_tab = session['proposal_data'].get('next_tab', 'defaultTabName')
    
    session.modified = True  # Ensure the session is marked as modified
    
    # Redirect back to the form page and include the next_tab as a query parameter
    return redirect(url_for('create_proposal', next_tab=next_tab))


@app.route('/insert', methods=['POST'])
def insert_item():
    insert_after = request.form['InsertAfter']
    part_num = request.form['PartNum']
    quantity = int(request.form['Quantity'])
    item = next((item for item in inventory if item['PartNum'] == part_num), None)
    if item:
        index = next((i for i, item in enumerate(session['bid_items']) if item['PartNum'] == insert_after), None)
        if index is not None:
            session['bid_items'].insert(index + 1, {**item, "Quantity": quantity, "Cost": float(item['Cost'])})
            session.modified = True  # Mark the session as modified to save changes
            total_cost = sum(item['Cost'] * item['Quantity'] for item in session['bid_items'])
            return jsonify(success=True, bid_items=session['bid_items'], total_cost=total_cost)
    return jsonify(success=False)

with app.app_context():
    db.create_all()

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [11/Mar/2024 11:07:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2024 11:07:05] "GET /inventory/manage HTTP/1.1" 404 -
127.0.0.1 - - [11/Mar/2024 11:07:09] "GET /factors/manage HTTP/1.1" 404 -
127.0.0.1 - - [11/Mar/2024 11:07:11] "GET /new-bid HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2024 11:07:12] "GET /inventory HTTP/1.1" 200 -
127.0.0.1 - - [11/Mar/2024 11:07:12] "GET /factors HTTP/1.1" 200 -


[{'Factor_ID': '100', 'Description': '1/2 LATERAL FIT 45%', 'Labor Hours': 0.0408}, {'Factor_ID': '1000', 'Description': '2" GALV FIT', 'Labor Hours': 0.05}, {'Factor_ID': '1000.1', 'Description': '2.5" GALV FIT', 'Labor Hours': 0.07}, {'Factor_ID': '1000.2', 'Description': '2" GALV FIT,VIT', 'Labor Hours': 0.05}, {'Factor_ID': '1001', 'Description': '1 1/2" GALV FIT', 'Labor Hours': 0.045}, {'Factor_ID': '1001.1', 'Description': '1 1/2" GALV FIT,VIT', 'Labor Hours': 0.042}, {'Factor_ID': '1002', 'Description': '1 1/4" GALV FITTINGS', 'Labor Hours': 0.033}, {'Factor_ID': '1002.1', 'Description': '1 1/4" GALV FIT,VIT', 'Labor Hours': 0.03}, {'Factor_ID': '1003', 'Description': '1" GALV. FIT.', 'Labor Hours': 0.03}, {'Factor_ID': '1003.1', 'Description': '1" GALV FIT,VIT', 'Labor Hours': 0.027}, {'Factor_ID': '1004', 'Description': '3/4" GALV. FIT.', 'Labor Hours': 0.0285}, {'Factor_ID': '1004.1', 'Description': '3/4"  GALV FIT,VIT', 'Labor Hours': 0.023}, {'Factor_ID': '1005', 'Descript